## Background

In this notebook I'll summarize the paper [Plan-and-Solve Prompting: Improving Zero-Shot Chain-of-Thought
Reasoning by Large Language Models](https://arxiv.org/pdf/2305.04091.pdf) (Lei Wang, et al) with the following questions. This exercise is part of a fastai study group assignment where we had multiple study group members present the different Chain-of-Thought papers.

This notebook has the following sections:

1) What is the problem which LLMs are failing to solve?
2) An overview of the prompting strategy
3) An example of the XoT prompt
4) Improvements on benchmarks
5) Does this prompt strategy strike you as useful? What are the downsides?
6) Test the prompt using a local model or API

## 1) What is the problem which LLMs are failing to solve?

Zero-shot-CoT (CoT = Chain-of-Thought) prompting involves appending the phrase _"Let's think step by step"_ to the end of a prompt and has [shown to improve LLM performance on reasoning tasks](https://arxiv.org/pdf/2205.11916.pdf) when compared to zero-shot prompting.

An example of a Zero-shot-CoT prompt:

> Q: After eating at the restaurant, Sally, Sam,
and Alyssa decided to divide the bill evenly.
If each person paid 45 dollars, what was the
total of the bill?  
>
> A: Let’s think step by step.


From the abstract of this paper (emphasis mine):

> Despite the success of Zero-shot-CoT, it still suffers from three pitfalls: **calculation errors, missing-step errors, and semantic misunderstanding errors**.

The authors address calculation errors and missing-step errors with Plan-and-Solve (PS) Prompting and PS+ Prompting. PS+ Prompting extends PS Prompting with more detailed instructions.

## 2) An overview of the prompting strategy

The prompting template for PS and PS+ prompting is fundamentally the same as Zero-shot-CoT

> Q: `[X]`.
>
> A: `[T]`.

Where `[X]` contains the input problem statement and `[T]` is a hand-crafted instruction.

The prompting strategy extends the Zero-shot-CoT prompt by adding more detailed instructions `[T]` on how the LLM should answer reasoning tasks to encourage the model to devise a plan, carry out the plan and lists intermediate steps.

### Plan-and-Solve (PS) Prompting

This prompting strategy replaces the Zero-shot-CoT prompt of _"Let's think step by step"_ with the following in order to address the issue of Zero-shot-CoT caused by missing reasoning steps:

> Q: `[X]`.
>
>A: Let's first understand the problem and devise a plan to solve the problem. Then, let's carry out the plan and solve the problem step by step.

### PS+ Prompting

This prompting strategy extends PS Prompting with the following instruction to address the calculation errors of Zero-shot-CoT:

> _"pay attention to calculation"_

and the following instruction to address missing-step errors:

> _"extract relevant variables and their corresponding numerals"_

It also includes the following instruction to enhance the LLMs ability to generate relevant and important reasoning steps:

> _"calculate intermediate results"_


An example of the full PS+ Prompting strategy:

> Q: `[X]`
>
> A: Let's first understand the problem, extract relevant variables and their corresponding numerals, and devise a plan. Then, let's carry out the plan, calculate intermediate results (pay attention to calculation and common sense), solve the problem step by step, and show the answer.

This prompting strategy can be easily customized to solve a variety of problems other than math reasoning.

## 3) An example of the XoT prompt

### PS Prompting

Using the same restaurant bill math problem as before, here is how it would be structured as a PS prompt:

> Q: After eating at the restaurant, Sally, Sam, and Alyssa decided to divide the bill evenly. If each person paid 45 dollars, what was the total of the bill?
>
> A: Let's first understand the problem and devise a plan to solve the problem. Then, let's carry out the plan and solve the problem step by step.

### PS+ Prompting

And here is how it would be structured with the more detailed PS+ prompt:

> Q: After eating at the restaurant, Sally, Sam, and Alyssa decided to divide the bill evenly. If each person paid 45 dollars, what was the total of the bill?
>
> A: Let's first understand the problem, extract relevant variables and their corresponding numerals, and devise a plan. Then, let's carry out the plan, calculate intermediate results (pay attention to calculation and common sense), solve the problem step by step, and show the answer.

## 4) Improvement on benchmarks

The authors evaluated their prompting on the following:

- 6 math reasoning datasets
  - **AQUA**: Algebraic word problems with natural language rationales.
  - **GSM8K**: High quality linguistically diverse grade school math word problems.
  - **MultiArith**: Math word problems requiring multiple reasoning steps and operations.
  - **AddSub**: Addition and subtraction arithmetic word problems.
  - **SingleEq**: Single-equation grade school algebra word problems with multiple math operations over non-negative rational numbers and one variable.
  - **SVAMP**: One-unknown arithmetic word problems for up-to-4 grade level students.
- 2 common sense reasoning datasets
  - **CommonsenseQA**: Multiple choice questions that require different types of commonsense knowledge to obtain the correct answers.
  - **StrategyQA**: Questions requiring multi-step reasoning but the reasoning steps are not given.
- 2 symbolic reasoning datasets
  - **Last Letter Concatenation**: Questions requiring the last letters of words in a name to be concatenated.
  - **Coin Flip**: Questions on whether a coin is still heads up after it is flipped or not flipped based on steps given in the questions.

### Arithmetic Reasoning Dataset Performance

![Math Reasoning Dataset Evaluation Results](table2.png)

Takeaways:

- PS and PS+ Prompting out-perform Zero-shot-CoT across all arithmetic reasoning datasets.
- PS+ Prompting out-performs Zero-shot-PoT on five out of six arithmetic datasets.
- PS Prompting out-performs Zero-shot-PoT on three arithmetic datasets.
- Average accuracy of PS+ slightly lower than Manual-CoT and slightly higher than Auto-CoT.


### Common Sense Reasoning Dataset Performance

<img src="table3.png" alt="Commonsense Reasoning Dataset Evaluation Results" style="width:50%;">

Takeaways:

- PS+ Prompting out-performs Zero-shot-CoT and underperforms versus Few-Shot-CoT

### Symbolic Reasoning Dataset Performance

<img src="table4.png" alt="Symbolic Reasoning Dataset Evaluation Results" style="width:50%;">

Takeaways:

- PS+ out-performs Zero-shot-CoT and Few-Shot-CoT on the Last Letter dataset.
- PS+ out-performs Zero-shot-CoT and underperforms compared to Few-Shot-CoT on the Coin Flip dataset.

### Addressing Issues of Zero-shot-CoT Prompting

At the start of the paper, the authors identify three issues with Zero-shot-CoT prompting: Calculation errors, Missing-step errors and Semantic misunderstanding errors. PS has fewer Missing-step and Semantic misunderstanding errors. PS+ has fewer Calculation and Missing-step errors.

<img src="errors.png" alt="Performance of Different Prompting Strategies on Calculation, Missing and Semantic Errors" style="width:50%;">

## 5) Does this prompt strategy strike you as useful? What are the downsides?

This prompt strategy seems somewhat useful. On one hand, it provides a framework for the LLM to respond with and performs better than zero-shot prompting in the paper's evaluation. On the other hand, for some of the models I tested, Zero-shot-CoT performed better than PS+ on a small set of 20 samples.

There are elements of the PS+ Prompting instruction that are also present in the system prompt that Jeremy recommended (emphasis mine):


> Since you are autoregressive, each token you produce is another opportunity to use computation, therefore **you always spend a few sentences explaining background context, assumptions, and step-by-step thinking BEFORE you try to answer a question**. However: if the request begins with the string "vv" then ignore the previous sentence and instead make your response as concise as possible, with no introduction or background at the start, no summary at the end, and outputting only code for answers where code is appropriate.  
>
> Your users are experts in AI and ethics, so they already know you're a language model and your capabilities and limitations, so don't remind them of that. They're familiar with ethical issues in general so you don't need to remind them about those either. Don't be verbose in your answers, but **do provide details and examples where it might help the explanation**. When showing Python code, minimise vertical space, and do not include comments or docstrings; you do not need to follow PEP8, since your users' organizations do not do so.

## 6) Test the prompt using a local model or api

In the paper's appendix, the authors provide two example PS+ prompts and corresponding outputs for each of the ten reasoning datasets they used. I tried out these given prompts on the following models, using HuggingChat and ChatGPT:

- HuggingFaceH4/zephyr-7b-alpha
- mistralai/Mistral-7B-Instruct-v0.1
- meta-llama/Llama-2-70b-chat-hf
- GPT-3.5



Here is a summary of accuracy of these models across the 20 samples:


||Zephyr|Mistral|Llama-2|GPT-3.5|
|:-:|:-:|:-:|:-:|:-:|
|**Zero-shot-CoT**|<u>65%</u>|60%|<u>70%</u>|80%|
|**PS+**|45%|<u>65%</u>|60%|<u>85%</u>|

<br>

I have linked the individual chats in the table below for reference (Z = Zero-shot-CoT). I wasn't able to test the Zephyr Zero-shot-CoT prompts before HuggingChat removed that model from that interface, and the shared chats in the Zephyr Chat HuggingFace Space expire after a few days, so I've documented the results for Zephyr (Z) in [this Google document](https://docs.google.com/document/d/1D5mhpN9dw3NXUn4Vc1KI4RzNFAkcex_CtpB1_mtjZUk/edit?usp=sharing).

<br>

- 0 = Incorrect
- 1 = Correct

<br>

|Dataset|Avg|Zephyr (Z)|Zephyr (PS+)|Mistral (Z)|Mistral (PS+)|Llama-2 (Z)|Llama-2 (PS+)|GPT-3.5 (Z)|GPT-3.5 (PS+)|
|:-|:-|:-|:-|:-|:-|:-|:-|:-|:-|
|AQuA|25%|0, 0|[0](https://hf.co/chat/r/zM3AJ_3), [0](https://hf.co/chat/r/plFbVxU)|[0](https://hf.co/chat/r/pEi9CzH), [0](https://hf.co/chat/r/pLqIr3u)|[0](https://hf.co/chat/r/ioJezfG), [0](https://hf.co/chat/r/x1bLxWh)|[0](https://hf.co/chat/r/QFRFn9p), [1](https://hf.co/chat/r/5u7k6dE)|[0](https://hf.co/chat/r/cApd104), [0](https://hf.co/chat/r/2zpSsCF)|[1](https://chat.openai.com/share/0dedf861-4628-463c-8901-90a21e887f82), [1](https://chat.openai.com/share/0990a57b-9dfe-47eb-acb2-eb648dfecb02)|[1](https://chat.openai.com/share/6f052c43-45d7-438e-98bf-add1573cf9bc), [0](https://chat.openai.com/share/731d81ad-a406-4cd8-ade5-4b7e97228bd1)|
|GSM8K|69%|1, 1|[0](https://hf.co/chat/r/fuwOr7W), [0](https://hf.co/chat/r/I5BGcYS)|[1](https://hf.co/chat/r/2w9nwS7), [0](https://hf.co/chat/r/L_fK1Fk)|[1](https://hf.co/chat/r/QulOu5P), [1](https://hf.co/chat/r/APItdpT)|[0](https://hf.co/chat/r/c-aXZP1), [1](https://hf.co/chat/r/4h-59fN)|[0](https://hf.co/chat/r/Y7Fivsi), [1](https://hf.co/chat/r/rJuuUBR)|[1](https://chat.openai.com/share/4cd70e29-ff18-4cb9-b188-1fbc9c48a204), [1](https://chat.openai.com/share/e712124d-3100-4483-96cc-64b817027435)|[1](https://chat.openai.com/share/3b83fed8-1f06-44c0-95bd-b399d756f3f2), [1](https://chat.openai.com/share/675c6dc3-29ad-4cc1-a54f-28a9bc541fdb)|
|MultiArith|56%|0, 1|[0](https://hf.co/chat/r/KmeC0ow), [0](https://hf.co/chat/r/gkErQYJ)|[1](https://hf.co/chat/r/gKtMIEB), [1](https://hf.co/chat/r/I6eW4qn)|[1](https://hf.co/chat/r/cKDU-Fx), [1](https://hf.co/chat/r/vP0zZG2)|[1](https://hf.co/chat/r/xPD-3qQ), [0](https://hf.co/chat/r/O7SWLEp)|[0](https://hf.co/chat/r/SvLTGtl), [0](https://hf.co/chat/r/8qlkQ9a)|[0](https://chat.openai.com/share/c04bade3-acd2-4f73-8d16-f9b452ab0fab), [1](https://chat.openai.com/share/5cdef4b6-7d11-485d-a07e-fdbd53cefeff)|[1](https://chat.openai.com/share/a7998a64-e387-4b0e-808a-a1009fd8be56), [1](https://chat.openai.com/share/3711e4a1-2a21-48bb-b013-62b8b18633fd)|
|SVAMP|88%|1, 1|[1](https://hf.co/chat/r/zQKHJ3l), [1](https://hf.co/chat/r/HQG-W9o)|[1](https://hf.co/chat/r/v5Unhi3), [1](https://hf.co/chat/r/_ZSSlWJ)|[1](https://hf.co/chat/r/OLBm0ct), [1](https://hf.co/chat/r/3l3Uwgj)|[1](https://hf.co/chat/r/MdORtPF), [0](https://hf.co/chat/r/s2q-k3s)|[1](https://hf.co/chat/r/iOajLWp), [0](https://hf.co/chat/r/X9elCAo)|[1](https://chat.openai.com/share/ce8f994a-8ff4-4a74-9b48-c7ccd05df4ba), [1](https://chat.openai.com/share/a86edc6f-97a3-4510-9819-a5f1d504870d)|[1](https://chat.openai.com/share/88aa9d4a-f675-493c-8857-1a7bde411f26), [1](https://chat.openai.com/share/0228aacc-9d78-47e2-b476-b9a4b2da7f2b)|
|AddSub|<u>94%</u>|1, 1|[1](https://hf.co/chat/r/HHseH12), [1](https://hf.co/chat/r/Y1Zog8Y)|[1](https://hf.co/chat/r/pygZ67S), [0](https://hf.co/chat/r/EI-XdIs)|[1](https://hf.co/chat/r/LOkpe9J), [1](https://hf.co/chat/r/3aCbK8b)|[1](https://hf.co/chat/r/la-Xv6E), [1](https://hf.co/chat/r/6NzqP0m)|[1](https://hf.co/chat/r/yAM421F), [1](https://hf.co/chat/r/sHPyX3X)|[1](https://chat.openai.com/share/983091a0-9c93-46e3-96e9-91ef14e725f1), [1](https://chat.openai.com/share/98b3e055-5a07-4d2d-a027-e86a836561c5)|[1](https://chat.openai.com/share/74a9f95a-1d7d-4b8e-808d-2220d721623f), [1](https://chat.openai.com/share/648b6287-b478-42fd-8ccb-12a549675745)|
|SingleEq|81%|1, 1|[1](https://hf.co/chat/r/XcUUALE), [1](https://hf.co/chat/r/rHbFPl-)|[1](https://hf.co/chat/r/FfXOnzY), [1](https://hf.co/chat/r/xvxgptS)|[0](https://hf.co/chat/r/U-LtaL0), [1](https://hf.co/chat/r/8Q7IWdQ)|[0](https://hf.co/chat/r/mR_J9or), [1](https://hf.co/chat/r/5l0SsQ5)|[0](https://hf.co/chat/r/a7HWg22), [1](https://hf.co/chat/r/J3OSbm5)|[1](https://chat.openai.com/share/0732c977-65e6-439a-bab7-8dba34741f43), [1](https://chat.openai.com/share/0b01b3e7-83c3-41d8-893b-e5d0ec2da160)|[1](https://chat.openai.com/share/ad32659e-b1e5-4193-9603-be5e7d44b107), [1](https://chat.openai.com/share/083bb423-fed6-4ad5-aeed-7967a2a76ccb)|
|CommonsenseQA|56%|1, 0|[1](https://hf.co/chat/r/wB0Zr8o), [0](https://hf.co/chat/r/aHyplu9)|[1](https://hf.co/chat/r/uwg3ML7), [0](https://hf.co/chat/r/GUta4fN)|[1](https://hf.co/chat/r/kBaVY6J), [0](https://hf.co/chat/r/vNBLOkf)|[1](https://hf.co/chat/r/t86EgAL), [0](https://hf.co/chat/r/m4IT419)|[1](https://hf.co/chat/r/qxBFyHz), [1](https://hf.co/chat/r/oqCYnxL)|[1](https://chat.openai.com/share/d86f92e1-2225-414b-ae10-65a67dd467b4), [0](https://chat.openai.com/share/40cfc746-6776-4ef6-8fdc-059f930a0570)|[1](https://chat.openai.com/share/d04155cf-8d70-4816-9102-23d8bcfef389), [0](https://chat.openai.com/share/16f96b90-0ac9-4b74-9206-de100f1d2725)|
|StrategyQA|56%|0, 1|[0](https://hf.co/chat/r/iDxT0pO), [0](https://hf.co/chat/r/vyC-LEd)|[0](https://hf.co/chat/r/ImTDLWk), [1](https://hf.co/chat/r/k6CyU2X)|[0](https://hf.co/chat/r/N3WqgU3), [1](https://hf.co/chat/r/tPE-FxN)|[1](https://hf.co/chat/r/MS-u84s), [1](https://hf.co/chat/r/9R6Iu9x)|[0](https://hf.co/chat/r/mVrnva3), [1](https://hf.co/chat/r/AuSw516)|[1](https://chat.openai.com/share/d7191b42-538b-4b4e-854a-eda5ea0dae9a), [0](https://chat.openai.com/share/7136b56d-27dc-44f1-ad23-96846966010c)|[1](https://chat.openai.com/share/45481ce8-384f-4094-8bab-0194cd9bced0), [1](https://chat.openai.com/share/c4f906cf-6a4d-45f2-bf26-da831a589d05)|
|Last Letters|38%|0, 0|[0](https://hf.co/chat/r/caDeIpj), [0](https://hf.co/chat/r/U7ZFkMU)|[0](https://hf.co/chat/r/SKnXfhL), [0](https://hf.co/chat/r/2AFI-8J)|[0](https://hf.co/chat/r/KyKlV5L), [0](https://hf.co/chat/r/S5R3TZA)|[1](https://hf.co/chat/r/s2cSASq), [1](https://hf.co/chat/r/wZTTaTQ)|[1](https://hf.co/chat/r/-tQ8DL6), [1](https://hf.co/chat/r/aGmbjUW)|[1](https://chat.openai.com/share/e6384359-c26e-4d9d-bc6e-66cdbecf523d), [0](https://chat.openai.com/share/0a974c18-f004-42ee-bad9-b52638ec26ea)|[1](https://chat.openai.com/share/c251d77c-1ff5-498f-883e-ff7af350cc96), [0](https://chat.openai.com/share/0565b6f5-0e8e-49b4-94bf-c8c0102c0c21)|
|Coin Flip|**100%**|1, 1|[1](https://hf.co/chat/r/j0qlCL3), [1](https://hf.co/chat/r/s6fvq44)|[1](https://hf.co/chat/r/RvNqC9Q), [1](https://hf.co/chat/r/4cachIM)|[1](https://hf.co/chat/r/9I7pyOe), [1](https://hf.co/chat/r/ELgmV-Y)|[1](https://hf.co/chat/r/TncZsxF), [1](https://hf.co/chat/r/QKptKiE)|[1](https://hf.co/chat/r/e0jhRvT), [1](https://hf.co/chat/r/rc2T8hh)|[1](https://chat.openai.com/share/d2282611-1d1c-43a1-9358-6992320dff80), [1](https://chat.openai.com/share/e7127b3a-085d-4932-9be8-ee054e73b804)|[1](https://chat.openai.com/share/f0be0db4-52f2-47ee-93fe-30cc526e9f35), [1](https://chat.openai.com/share/d35b9041-4045-4f23-9056-fe478d1ce19d)|



I hope you enjoyed this blog post!